<a href="https://colab.research.google.com/github/PM-Inno/Classification-Hackathon/blob/master/ZindiCompetition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pprint import pprint
import matplotlib.pyplot as plt
from matplotlib.pyplot import *
import seaborn as sns

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, f1_score



from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn import svm
import lightgbm as lgb
from lightgbm import LGBMClassifier
import xgboost as xgb






In [134]:
train = pd.read_csv('training.csv')
test = pd.read_csv('test.csv')
Var_def = pd.read_csv('Xente_Variable_Definitions.csv')
sample_sub = pd.read_csv('sample_submission.csv')
test.head()  

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy
0,TransactionId_50600,BatchId_35028,AccountId_2441,SubscriptionId_4426,CustomerId_2857,UGX,256,ProviderId_5,ProductId_3,airtime,ChannelId_3,1000.0,1000,2019-02-13T10:01:40Z,4
1,TransactionId_95109,BatchId_45139,AccountId_3439,SubscriptionId_2643,CustomerId_3874,UGX,256,ProviderId_5,ProductId_15,financial_services,ChannelId_3,2000.0,2000,2019-02-13T10:02:12Z,2
2,TransactionId_47357,BatchId_74887,AccountId_4841,SubscriptionId_3829,CustomerId_2857,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-50.0,50,2019-02-13T10:02:30Z,2
3,TransactionId_28185,BatchId_11025,AccountId_2685,SubscriptionId_4626,CustomerId_3105,UGX,256,ProviderId_5,ProductId_10,airtime,ChannelId_3,3000.0,3000,2019-02-13T10:02:38Z,4
4,TransactionId_22140,BatchId_29804,AccountId_4841,SubscriptionId_3829,CustomerId_3105,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-60.0,60,2019-02-13T10:02:58Z,2


In [0]:
train_df = train.select_dtypes(exclude=['O'])
test_df = test.select_dtypes(exclude=['O'])

In [136]:
print(train.FraudResult.value_counts())

0    95469
1      193
Name: FraudResult, dtype: int64


In [137]:
y = train_df.FraudResult
X = train_df.drop('FraudResult', axis=1)

# setting up testing and training sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=27)



dummy = DummyClassifier(strategy='most_frequent').fit(X_train, y_train)
dummy_pred = dummy.predict(X_test)

# checking unique labels
print('Unique predicted labels: ', (np.unique(dummy_pred)))

# checking accuracy
print('Test score: ', f1_score(y_test, dummy_pred))

Unique predicted labels:  [0]
Test score:  0.0


In [138]:
lr = LogisticRegression(solver='liblinear').fit(X_train, y_train)
 
# Predict on training set
lr_pred = lr.predict(X_test)


accuracy_score(y_test, lr_pred)


predictions = pd.DataFrame(lr_pred)
predictions[0].value_counts()

print(f1_score(y_test, lr_pred))

0.40476190476190477


In [139]:
from sklearn.ensemble import RandomForestClassifier


rfc = RandomForestClassifier(n_estimators=10).fit(X_train, y_train)

# predict on test set
rfc_pred = rfc.predict(X_test)

print(accuracy_score(y_test, rfc_pred))

print(f1_score(y_test, rfc_pred))

0.999581869877906
0.8936170212765957


In [140]:
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier(n_estimators=200,learning_rate=0.15, random_state=0).fit(X_train,y_train)
clf_pred = clf.predict(X_test)

print(accuracy_score(y_test, clf_pred))

print(f1_score(y_test, clf_pred))

0.9994564308412778
0.8571428571428571


In [141]:
clf2 = AdaBoostClassifier(n_estimators=100, random_state=0).fit(X_train,y_train)
# predict on test set
clf_pred = clf2.predict(X_test)

print(accuracy_score(y_test, clf_pred))

print(f1_score(y_test, clf_pred))

0.9995400568656966
0.8910891089108911


In [142]:
from sklearn.utils import resample

# Separate input features and target
y = train_df.FraudResult
X = train_df.drop('FraudResult', axis=1)

# setting up testing and training sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=27)

X = pd.concat([X_train, y_train], axis=1)
X.head()


not_fraud = X[X.FraudResult==0]
fraud = X[X.FraudResult==1]

# upsample minority
fraud_upsampled = resample(fraud,
                          replace=True, # sample with replacement
                          n_samples=len(not_fraud), # match number in majority class
                          random_state=27) # reproducible results

# combine majority and upsampled minority
upsampled = pd.concat([not_fraud, fraud_upsampled])

# check new class counts
upsampled.FraudResult.value_counts()


y_train = upsampled.FraudResult
X_train = upsampled.drop('FraudResult', axis=1)

upsampled = LogisticRegression(solver='liblinear').fit(X_train, y_train)

upsampled_pred = upsampled.predict(X_test)

print(accuracy_score(y_test, upsampled_pred))
print(f1_score(y_test, upsampled_pred))

0.9899230640575347
0.29325513196480935


In [143]:
not_fraud_downsampled = resample(not_fraud,
                                replace = False, # sample without replacement
                                n_samples = len(fraud), # match minority n
                                random_state = 27) # reproducible results

# combine minority and downsampled majority
downsampled = pd.concat([not_fraud_downsampled, fraud])

# checking counts
downsampled.FraudResult.value_counts()


y_train = downsampled.FraudResult
X_train = downsampled.drop('FraudResult', axis=1)

#undersampled = LogisticRegression(solver='liblinear').fit(X_train, y_train
undersampled = RandomForestClassifier(n_estimators=10).fit(X_train, y_train)


undersampled_pred = undersampled.predict(X_test)
print(accuracy_score(y_test, undersampled_pred))
print(f1_score(y_test, undersampled_pred))

0.976668339187155
0.1545454545454545


In [144]:
from imblearn.over_sampling import SMOTE

# Separate input features and target
y = train_df.FraudResult
X = train_df.drop('FraudResult', axis=1)
# setting up testing and training sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=27)

sm = SMOTE(random_state=27, ratio=1.0)
X_train, y_train = sm.fit_sample(X_train, y_train)

#smote = LogisticRegression(solver='liblinear').fit(X_train, y_train)
smote = RandomForestClassifier(n_estimators=10).fit(X_train, y_train)

smote_pred = smote.predict(X_test)

# Checking accuracy
print(accuracy_score(y_test, smote_pred))

print(f1_score(y_test, smote_pred))

0.9949406255226626
0.431924882629108


In [0]:
###Test data

#lr_test = lr_pred.predict(test_df) 
rfc_test = rfc.predict(test_df)
upsampled_test = upsampled.predict(test_df)
smote_test = smote.predict(test_df)
clf2_test = clf2.predict(test_df)

In [0]:
sub = pd.DataFrame()
sub['TransactionId'] = test['TransactionId']
sub['FraudResult'] = clf2_test
sub.to_csv('submission3.csv',index=False)



References

1. https://towardsdatascience.com/methods-for-dealing-with-imbalanced-data-5b761be45a18

2. https://www.kaggle.com/gpreda/credit-card-fraud-detection-predictive-models